# Lab 20: Sigma Rule Fundamentals

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab20_sigma_fundamentals.ipynb)

Learn Sigma rule syntax and use AI to generate detection rules from threat descriptions.

## Learning Objectives
- Understand Sigma rule structure and syntax
- Convert threat descriptions to Sigma rules
- Use LLMs to generate and validate detection rules
- Convert Sigma to SIEM-specific queries (Splunk, Elastic, etc.)

## What is Sigma?

Sigma is a generic signature format for log events. Think of it as "YARA for logs."

```yaml
title: Suspicious PowerShell Download
status: experimental
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains|all:
            - 'powershell'
            - 'downloadstring'
    condition: selection
level: high
```

**Next:** Lab 22 (Vulnerability Scanner AI)

In [ ]:
#@title Install dependencies (Colab only)
%pip install -q anthropic openai google-generativeai python-dotenv pyyaml

In [ ]:
#@title LLM Setup (Provider-Agnostic)
import os
import yaml

try:
    from google.colab import userdata
    for key in ["ANTHROPIC_API_KEY", "OPENAI_API_KEY", "GOOGLE_API_KEY"]:
        try:
            os.environ[key] = userdata.get(key)
        except:
            pass
except:
    pass

def setup_llm():
    if os.environ.get("ANTHROPIC_API_KEY"):
        return "anthropic", "claude-sonnet-4.5"
    elif os.environ.get("OPENAI_API_KEY"):
        return "openai", "gpt-5"
    elif os.environ.get("GOOGLE_API_KEY"):
        return "google", "gemini-3-flash"
    raise ValueError("No API key found.")

def query_llm(prompt: str, system: str = "You are a security detection engineer.") -> str:
    provider, model = setup_llm()
    if provider == "anthropic":
        from anthropic import Anthropic
        client = Anthropic()
        response = client.messages.create(
            model=model, max_tokens=2048, system=system,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.content[0].text
    elif provider == "openai":
        from openai import OpenAI
        client = OpenAI()
        response = client.chat.completions.create(
            model=model, max_tokens=2048,
            messages=[{"role": "system", "content": system}, {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    elif provider == "google":
        import google.generativeai as genai
        genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
        model_instance = genai.GenerativeModel(model)
        response = model_instance.generate_content(f"{system}\n\n{prompt}")
        return response.text

provider, model = setup_llm()
print(f"✅ Using {provider} ({model})")

## Sigma Rule Structure

Every Sigma rule has these components:

| Field | Required | Description |
|-------|----------|-------------|
| title | Yes | Short description |
| logsource | Yes | What logs to search |
| detection | Yes | The actual detection logic |
| status | No | experimental, test, stable |
| level | No | informational, low, medium, high, critical |

In [ ]:
# Example Sigma rule
EXAMPLE_SIGMA = """
title: Mimikatz Command Line
id: a642964e-bead-4bed-8910-1bb4d63e3b4d
status: test
description: Detects Mimikatz command line patterns
author: AI Security Lab
logsource:
    category: process_creation
    product: windows
detection:
    selection_cmd:
        CommandLine|contains:
            - 'sekurlsa::'
            - 'kerberos::'
            - 'crypto::'
            - 'lsadump::'
    selection_img:
        Image|endswith: '\\mimikatz.exe'
    condition: selection_cmd or selection_img
falsepositives:
    - Legitimate security tools
level: critical
tags:
    - attack.credential_access
    - attack.t1003
"""

# Parse and display
rule = yaml.safe_load(EXAMPLE_SIGMA)
print("📋 Sigma Rule Structure:")
for key, value in rule.items():
    print(f"  {key}: {value}")

## Using AI to Generate Sigma Rules

In [ ]:
SIGMA_PROMPT = """
You are a detection engineer. Generate a Sigma rule for the following threat:

{threat_description}

Requirements:
1. Use valid Sigma YAML syntax
2. Include appropriate logsource (category, product)
3. Use meaningful field modifiers (contains, endswith, etc.)
4. Set appropriate level (low/medium/high/critical)
5. Add MITRE ATT&CK tags if applicable

Return ONLY the YAML rule, no explanation.
"""

def generate_sigma_rule(threat_description: str) -> str:
    """Generate a Sigma rule from a threat description."""
    prompt = SIGMA_PROMPT.format(threat_description=threat_description)
    response = query_llm(prompt)
    
    # Extract YAML from response (handle markdown code blocks)
    if "```yaml" in response:
        response = response.split("```yaml")[1].split("```")[0]
    elif "```" in response:
        response = response.split("```")[1].split("```")[0]
    
    return response.strip()

# Test it
threat = """Detect when certutil.exe is used to download files from the internet.
This is a common LOLBin technique (T1105 - Ingress Tool Transfer)."""

print("🎯 Generating Sigma rule for:")
print(threat)
print("\n" + "="*60 + "\n")

rule = generate_sigma_rule(threat)
print(rule)

In [ ]:
def validate_sigma_rule(rule_text: str) -> dict:
    """Validate a Sigma rule's structure."""
    try:
        rule = yaml.safe_load(rule_text)
        
        required_fields = ['title', 'logsource', 'detection']
        missing = [f for f in required_fields if f not in rule]
        
        if missing:
            return {"valid": False, "error": f"Missing required fields: {missing}"}
        
        if 'condition' not in rule.get('detection', {}):
            return {"valid": False, "error": "Detection missing 'condition'"}
        
        return {"valid": True, "rule": rule}
    except yaml.YAMLError as e:
        return {"valid": False, "error": f"YAML parse error: {e}"}

# Validate the generated rule
validation = validate_sigma_rule(rule)
if validation["valid"]:
    print("✅ Rule is valid!")
else:
    print(f"❌ Validation error: {validation['error']}")

## Converting Sigma to SIEM Queries

Sigma rules can be converted to various SIEM formats.

In [ ]:
CONVERSION_PROMPT = """
Convert this Sigma rule to a {target_format} query:

```yaml
{sigma_rule}
```

Return ONLY the query, no explanation.
"""

def convert_sigma(sigma_rule: str, target_format: str) -> str:
    """Convert Sigma rule to target SIEM format."""
    prompt = CONVERSION_PROMPT.format(
        sigma_rule=sigma_rule,
        target_format=target_format
    )
    return query_llm(prompt)

# Convert to different formats
print("📊 Converting to Splunk SPL:")
print(convert_sigma(rule, "Splunk SPL"))

print("\n" + "="*60 + "\n")

print("📊 Converting to Elastic KQL:")
print(convert_sigma(rule, "Elastic KQL"))

## Exercises

### Exercise 1: Generate rules for common attacks
Create Sigma rules for:
- PowerShell encoded command execution
- Scheduled task creation for persistence
- LSASS memory dumping

### Exercise 2: Build a rule library
Create a function that generates multiple rules from a list of MITRE techniques.

### Exercise 3: Rule quality scoring
Use an LLM to score the quality of generated rules (specificity, false positive rate, etc.)

## Next Steps

- **Lab 21**: Generate YARA rules for malware detection
- **Lab 23**: Build a detection pipeline using these rules
- **Lab 12**: Test your rules with purple team simulations